<a href="https://colab.research.google.com/github/LeeSeungwon89/Python_for_Data_Analytics_Science/blob/main/4.%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 데이터 전처리하기**

데이터를 분석하기 전에 데이터 품질을 제고하는 데이터 전처리(preprocessing) 작업을 선행해야 합니다. 데이터 분석 결과가 왜곡되는 상황을 방지하고 데이터 분석 정확도를 제고하기 위한 필수 작업입니다. 

데이터 분석 강의, 인터넷 정보, 서적, **파이썬 머신러닝 판다스 데이터 분석(오승환, 정보문화사)**을 참고했습니다.

## **1.1. 누락된 데이터**

특정 데이터를 실수로 입력하지 않거나 파일 형식을 변환하면서 데이터가 손상되는 경우가 많습니다. 전처리 작업을 거치지 않으면 정확한 데이터 분석 결과를 얻기 어렵습니다. 

### **1.1.1 누락된 데이터 확인하기**

데이터 분석을 위한 라이브러리 목록입니다. 복사해서 사용합니다.

In [71]:
# 기본적인 라이브러리입니다.
import time
import random
import math

# 데이터 분석을 위한 라이브러리입니다.
import numpy as np
import pandas as pd
import sklearn as sk

# 수학 라이브러리입니다.
import scipy as sp
import statsmodels.api as sm

# 웹 스크레이핑을 위한 라이브러리입니다
import re
import requests
from bs4 import BeautifulSoup
import os
import json

# 시각화 라이브러리입니다.
import matplotlib as mpl
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 자료를 바로 띄워줍니다.
%matplotlib inline

# 그래프에 retina를 지정합니다.
%config InlineBackend.figure_format = 'retina'

# 음수 부호를 깨지지 않도록 합니다.
mpl.rc('axes', unicode_minus=False)

사용할 데이터셋은 `seaborn` 라이브러리의 타이타닉 탑승객 정보 데이터셋입니다.

In [72]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


`info()` 메서드만으로도 누락된 데이터를 어느 정도 파악할 수 있습니다. 총 데이터 개수는 891인데 'age', 'embarked', 'deck', 'embark_town' 열은 데이터 수가 891 미만으로 모자랍니다.

참고로 누락된 데이터가 `NaN`으로 잡히지 않는 경우도 있습니다. 특정 기호인 `?`, `!`, `-`, ` ` 등으로 이루어진 값은 `NaN`으로 잡히지 않습니다. 이 경우 `replace()` 메서드와 넘파이의 `np.nan`를 혼용하여 `NaN`으로 바꿀 필요가 있습니다. `*`로 채워진 경우라면 `df.replace('*', np.nan, inplace=True)` 형식으로 바꿀 수 있습니다.

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [74]:
df.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


`isnull()` 메서드를 사용하면 `NaN`을 의미하는 `True`가 1, `False`가 0입니다. 이 값을 열 기준으로 모두 더하는 `sum(axis=0)` 메서드를 활용하면 데이터가 누락된 열 개수를 파악할 수 있습니다. `axis=0`과 `axis=1`은 혼동하기 쉬운 개념입니다. 명확하게 정리하겠습니다.

- `axis=0`: 행(인덱스) 방향으로 작동합니다. 책을 위로 쌓으면서 정리한다고 볼 수 있습니다. 다시 말하면 각 열의 모든 행으로 작동합니다. 작업 결과는 행으로 출력됩니다. 

- `axis=1`: 열(컬럼) 방향으로 작동합니다. 책을 옆으로 정리한다고 볼 수 있습니다. 다시 말하면 각 행의 모든 열로 작동합니다. 작업 결과는 열로 출력됩니다. 

In [75]:
df.head().isnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [76]:
df.isnull().sum(axis=0)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

`notnull()` 메서드는 `isnull()` 메서드의 반대 방향으로 작동합니다.

In [77]:
df.head().notnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True


In [78]:
df.notnull().sum(axis=0)

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
dtype: int64

`value_counts()` 메서드는 각 열의 데이터 값마다 개수로 출력합니다. `dropna=False`로 지정하면 `NaN`의 개수도 함께 출력합니다.

In [79]:
df['age'].value_counts(dropna=False)

NaN      177
24.00     30
22.00     27
18.00     26
28.00     25
        ... 
36.50      1
55.50      1
66.00      1
23.50      1
0.42       1
Name: age, Length: 89, dtype: int64

In [80]:
df['deck'].value_counts(dropna=False)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

반목문으로도 `NaN`을 확인할 수 있습니다. 예외 처리 구문은 [링크](https://wikidocs.net/30)를 참조하시기 바랍니다.

In [81]:
df_nan = df.isnull()

for column in df_nan:
    count_nan = df_nan[column].value_counts()
    try:
        print(f'{column}: {count_nan[True]}')
    # `missing_count`의 데이터에 `True`가 없으면 예외로 처리합니다.
    except:
        print(f'{column}: 0')

survived: 0
pclass: 0
sex: 0
age: 177
sibsp: 0
parch: 0
fare: 0
embarked: 2
class: 0
who: 0
adult_male: 0
deck: 688
embark_town: 2
alive: 0
alone: 0


### **1.1.2. 누락된 데이터 제거하기**

누락된 원소가 있는 행(레코드)을 삭제하거나 열(특성)을 삭제하는 방법을 취할 수 있습니다.

'deck' 열은 누락된 원소가 688개이므로 `dropna()` 메서드를 사용하여 열을 통으로 삭제해야 합니다. `axis=0`으로 지정하여 행을 기준으로 삼아 아래로 카운트하고, `thresh=300`으로 지정하여 `NaN`이 300개 이상 카운트된 열을 삭제하겠습니다.

In [82]:
df_thresh = df.dropna(axis=0, thresh=300)
print(df_thresh.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')


'deck' 열만 삭제되었습니다.

'age' 열의 원소는 177개만큼 `NaN`입니다. 데이터를 분석할 때 'age'가 중요한 특성(열, 변수)라면 특성은 남겨두고 `NaN`인 행(레코드)를 제거하는 방법을 취할 수 있습니다. `subset=['age']` 옵션은 'age' 열만 한정한다는 의미이고, `how='any'`(기본값) 옵션은 `NaN`이 하나라도 있으면 삭제한다는 의미입니다. `how='all'` 옵션은 모든 값이 `Nan`이면 삭제한다는 의미입니다.

In [83]:
df_age = df.dropna(subset=['age'], how='any', axis=0)
len(df_age)

714

### **1.1.3. 누락된 데이터 대체하기**

기존 데이터를 최대한 살려서 사용할 수 있도록 누락된 값을 다른 값으로 대체하는 방법을 우선 고려해야 합니다. 무조건 행이나 열을 삭제하는 방법은 데이터를 크게 손상시킵니다.

핵심적으로 쓰이는 메서드는 `fillna()`입니다. `NaN` 값을 다른 값으로 대체하는 메서드입니다. 누락된 값을 대체할 값으로는 **평균값**, **최빈값**, **중간값** 등을 고려할 수 있습니다.

먼저 'age'열의 `NaN` 값을 평균값으로 대체하겠습니다.

In [84]:
age_mean = df['age'].mean(axis=0)
# `inplace=True` 옵션을 지정하여 원본 데이터프레임 객체를 영구적으로 변경합니다.
df['age'].fillna(age_mean, inplace=True)

print(f"NaN: {sum(df['age'].isnull())}개")
print()
print(df['age'].value_counts(dropna=False))

NaN: 0개

29.699118    177
24.000000     30
22.000000     27
18.000000     26
28.000000     25
            ... 
55.500000      1
53.000000      1
20.500000      1
23.500000      1
0.420000       1
Name: age, Length: 89, dtype: int64


`NaN` 값 177개가 평균값인 29.699로 대체되었습니다.

다음은 'embark_town' 열의 `NaN` 값을 최빈값으로 대체하겠습니다. 먼저 `NaN` 값이 존재하는 레코드를 확인하겠습니다. 불리언 인덱싱을 사용합니다.

In [85]:
embark_town_nan = df['embark_town'].isnull()
df[embark_town_nan]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
61,1,1,female,38.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True
829,1,1,female,62.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True


`value_counts()` 메서드로 각 값의 개수를 구합니다.

In [86]:
embark_town_value_counts = df['embark_town'].value_counts(dropna=False)
embark_town_value_counts

Southampton    644
Cherbourg      168
Queenstown      77
NaN              2
Name: embark_town, dtype: int64

최빈값은 'Southampton'입니다. `idxmax()` 메서드를 사용하면 최빈값만 출력할 수 있습니다.

In [87]:
embark_town_idxmax = embark_town_value_counts.idxmax()
embark_town_idxmax

'Southampton'

`NaN` 값을 최빈값인 'Southampton'로 대체하고 대체 여부를 확인하겠습니다.

In [88]:
df['embark_town'].fillna(embark_town_idxmax, inplace=True)
df.loc[[61, 829], ['embark_town']]

,embark_town
61,Southampton
829,Southampton


아래 형식으로도 확인할 수도 있지만 가능하면 `loc`과 `iloc`을 사용하는 습관을 들이는 것이 좋습니다.

In [89]:
df['embark_town'][[61, 829]]

61     Southampton
829    Southampton
Name: embark_town, dtype: object

서로 이웃한 값은 유사한 특성을 가졌을 가능성이 높습니다(물론 데이터셋마다 차이가 있습니다). 이 경우 `NaN` 값을 이웃한 값으로 바꾸려면 `fillna()` 메서드에 `method='bfill'` 또는 `method='ffill'` 옵션을 지정합니다. 아래는 이 옵션에 대한 예시입니다. 먼저 새로운 데이터프레임을 생성하겠습니다.

In [90]:
df_ex = pd.DataFrame(np.random.randn(5, 3), columns=['C1', 'C2', 'C3'])

df_ex.iloc[0, 0] = None
df_ex.loc[1, ['C1', 'C3']] = np.nan
df_ex.loc[2, 'C2'] = np.nan
df_ex.loc[3, 'C2'] = np.nan
df_ex.loc[4, 'C3'] = np.nan

df_ex

,C1,C2,C3
0,NaN,0.387830,-1.804365
1,NaN,1.517328,NaN
2,-0.361229,NaN,2.025958
3,0.621519,NaN,0.182908
4,1.632478,0.368058,NaN


`ffill`을 적용합니다. `NaN` 값에 `NaN` 값의 바로 위 레코드의 값을 채웁니다.

In [91]:
df_ex.fillna(method='ffill')

,C1,C2,C3
0,NaN,0.387830,-1.804365
1,NaN,1.517328,-1.804365
2,-0.361229,1.517328,2.025958
3,0.621519,1.517328,0.182908
4,1.632478,0.368058,0.182908


바로 위 레코드가 없는 `(0, 'C1')`, `(1, 'C1')`는 그대로 `NaN`입니다.

`bfill`을 적용합니다. `NaN` 값에 `NaN` 값의 바로 아래 레코드의 값을 채웁니다.

In [92]:
df_ex.fillna(method='bfill')

,C1,C2,C3
0,-0.361229,0.387830,-1.804365
1,-0.361229,1.517328,2.025958
2,-0.361229,0.368058,2.025958
3,0.621519,0.368058,0.182908
4,1.632478,0.368058,NaN


## **1.2. 중복된 데이터**

데이터셋에 동일한 레코드가 있으면 삭제해야 합니다.

### **1.2.1. 중복된 데이터 확인하기**

먼저 중복된 레코드를 가진 임의의 데이터프레임을 생성하겠습니다.

In [93]:
df = pd.DataFrame({
    'c1': ['a', 'a', 'b', 'b', 'c'],
    'c2': [1, 1, 2, 3, 4],
    'c3': [1, 1, 2, 2, 4]
    })
df

,c1,c2,c3
0,a,1,1
1,a,1,1
2,b,2,2
3,b,3,2
4,c,4,4


중복된 레코드를 확인하려면 데이터프레임 자체에 `duplicated()` 메서드를 적용합니다. 이 메서드는 중복된 레코드에 해당하는 위치를 `True`로 반환합니다. 

In [94]:
df_dupl = df.duplicated()
df_dupl

0    False
1     True
2    False
3    False
4    False
dtype: bool

0행과 1행의 레코드는 중복된 레코드입니다. 0행의 레코드는 첫 레코드이므로 정상 레코드로 인식하여 `False`를 반환하지만, 첫 레코드와 중복된 1행의 레코드는 `True`를 반환하는 것입니다.

중복된 레코드 개수는 `sum()` 메서드를 활용해서 출력할 수 있습니다.

In [95]:
df_dupl.sum()

1

이번에는 열만 골라서 중복 여부를 살피겠습니다. 처음 나온 값은 `False`, 처음 나온 값과 중복된 값은 `True`입니다.

In [96]:
col_dupl = df['c1'].duplicated()
col_dupl

0    False
1     True
2    False
3     True
4    False
Name: c1, dtype: bool

### **1.2.2. 중복된 데이터 제거하기**

`drop_duplicates()` 메서드를 사용하면 고유한 데이터를 남기고 중복되는 데이터를 제거할 수 있습니다.

In [97]:
df

,c1,c2,c3
0,a,1,1
1,a,1,1
2,b,2,2
3,b,3,2
4,c,4,4


In [98]:
# `inplace=True`로 지정하면 기존 데이터프레임 자체를 변경합니다.
# 여기서는 지정하지 않겠습니다.
df.drop_duplicates()

,c1,c2,c3
0,a,1,1
2,b,2,2
3,b,3,2
4,c,4,4


1행의 레코드가 삭제되었습니다.

열에 적용하겠습니다.

In [99]:
df.drop_duplicates('c1')

,c1,c2,c3
0,a,1,1
2,b,2,2
4,c,4,4


In [100]:
# `subset=['c1', 'c3']` 옵션과 같습니다.
df.drop_duplicates(['c1', 'c3'])

,c1,c2,c3
0,a,1,1
2,b,2,2
4,c,4,4


## **1.3. 데이터 표준화**

한 데이터셋에 여러 형태와 방식(단위, 대소문자, 약칭 등)으로 데이터가 구성되었다면 데이터 분석 정확도는 떨어질 수 밖에 없습니다. 데이터 일관성을 확보하려면 반드시 데이터 표준화 작업을 거쳐야 합니다.

### **1.3.1. 단위 환산하기**

In [101]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


[UCI 자동차 연비 데이터셋](https://archive.ics.uci.edu/ml/datasets/auto+mpg)을 사용하겠습니다.

In [102]:
df = pd.read_csv('/content/gdrive/MyDrive/Python_for_Data_Analytics_Science/5674-833_4th/part5/auto-mpg.csv',
                 header=None)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model year', 'origin', 'name']
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


`NaN` 값이 있는지 확인합니다.

In [104]:
df.isnull().sum(axis=0)

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
name            0
dtype: int64

중복된 레코드가 있는지 여부를 확인합니다.

In [105]:
df.duplicated().sum()

0

'mpg' 열의 mpg는 mile per gallon(갤런당 마일)을 의미합니다. kilometer per liter(리터당 킬로미터)로 변환하기 위해 'kpl' 열을 추가하고 mpg 단위를 kpl 단위로 환산하겠습니다.

참고로 1mile은 1.60934km이고, 1gallon은 3.78541L이므로, 1mpg는 0.425km/L입니다.

In [106]:
kpl_from_mpg = 1.60934 / 3.78541
df['kpl'] = (df['mpg'] * kpl_from_mpg).round(3)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.653
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.377
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.653
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,6.802
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,7.227


### **1.3.2. 자료형 변환하기**

숫자형 원소가 문자열 형식으로 입력되었다면 반드시 변환하는 과정을 거쳐야 합니다. 데이터 형식을 확인하겠습니다.

In [107]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.653
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.377
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.653
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,6.802
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,7.227


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
 9   kpl           398 non-null    float64
dtypes: float64(5), int64(3), object(2)
memory usage: 31.2+ KB


`dtypes` 속성으로 바로 접근할 수도 있습니다. 참고로 객체의 속성을 확인하려면 `dir()` 메서드를 사용합니다.

In [109]:
dir(df)[:10]

['T',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '__abs__',
 '__add__',
 '__and__']

In [110]:
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model year        int64
origin            int64
name             object
kpl             float64
dtype: object

'horsepower'는 엔진 출력을 나타내는 값을 담은 열입니다. 문자열 타입이므로 숫자형으로 변환해야 합니다. 다만 이 열에 숫자로 입력된 값이 문자열 타입인 이유를 먼저 파악할 필요가 있습니다. `unique()` 메서드으로 이 열의 고유값을 출력하고, `nunique()` 메서드로 고유값 개수를 출력하겠습니다.

In [111]:
print(df['horsepower'].unique())
print()
print(f'고유값 개수는 {df["horsepower"].nunique(dropna=False)}개입니다.')

['130.0' '165.0' '150.0' '140.0' '198.0' '220.0' '215.0' '225.0' '190.0'
 '170.0' '160.0' '95.00' '97.00' '85.00' '88.00' '46.00' '87.00' '90.00'
 '113.0' '200.0' '210.0' '193.0' '?' '100.0' '105.0' '175.0' '153.0'
 '180.0' '110.0' '72.00' '86.00' '70.00' '76.00' '65.00' '69.00' '60.00'
 '80.00' '54.00' '208.0' '155.0' '112.0' '92.00' '145.0' '137.0' '158.0'
 '167.0' '94.00' '107.0' '230.0' '49.00' '75.00' '91.00' '122.0' '67.00'
 '83.00' '78.00' '52.00' '61.00' '93.00' '148.0' '129.0' '96.00' '71.00'
 '98.00' '115.0' '53.00' '81.00' '79.00' '120.0' '152.0' '102.0' '108.0'
 '68.00' '58.00' '149.0' '89.00' '63.00' '48.00' '66.00' '139.0' '103.0'
 '125.0' '133.0' '138.0' '135.0' '142.0' '77.00' '62.00' '132.0' '84.00'
 '64.00' '74.00' '116.0' '82.00']

고유값 개수는 94개입니다.


'?'가 포함되어 있기 때문에 숫자형이 아닌 문자열 타입으로 구성된 것입니다. 이 상황에서는 먼저 모든 '?'를 지우고 `NaN` 값으로 치환하는 방법을 시도할 수 있습니다.

치환하기 전에 먼저 '?'가 포함된 레코드 개수를 파악하겠습니다. `value_counts()` 메서드의 출력물은 시리즈 형태이므로 리스트 인덱싱을 통해 접근할 수 있습니다.

In [112]:
df['horsepower'].value_counts()['?']

6

이제 '?' 6개를 `NaN` 값으로 치환하고, `NaN` 값이 포함된 레코드 6개를 제거하겠습니다.

In [113]:
df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], inplace=True)

print(df['horsepower'].unique())
print()
print(f'고유값 개수는 {df["horsepower"].nunique(dropna=False)}개입니다.')
print()
print(df.info())

['130.0' '165.0' '150.0' '140.0' '198.0' '220.0' '215.0' '225.0' '190.0'
 '170.0' '160.0' '95.00' '97.00' '85.00' '88.00' '46.00' '87.00' '90.00'
 '113.0' '200.0' '210.0' '193.0' '100.0' '105.0' '175.0' '153.0' '180.0'
 '110.0' '72.00' '86.00' '70.00' '76.00' '65.00' '69.00' '60.00' '80.00'
 '54.00' '208.0' '155.0' '112.0' '92.00' '145.0' '137.0' '158.0' '167.0'
 '94.00' '107.0' '230.0' '49.00' '75.00' '91.00' '122.0' '67.00' '83.00'
 '78.00' '52.00' '61.00' '93.00' '148.0' '129.0' '96.00' '71.00' '98.00'
 '115.0' '53.00' '81.00' '79.00' '120.0' '152.0' '102.0' '108.0' '68.00'
 '58.00' '149.0' '89.00' '63.00' '48.00' '66.00' '139.0' '103.0' '125.0'
 '133.0' '138.0' '135.0' '142.0' '77.00' '62.00' '132.0' '84.00' '64.00'
 '74.00' '116.0' '82.00']

고유값 개수는 93개입니다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    

레코드 수가 398개에서 392개로 줄었습니다.

이제 'horsepower' 열의 타입을 숫자형으로 바꾸겠습니다. `astype()` 메서드를 활용합니다. 형 변환 후에는 반드시 객체로 넘겨줘야 영구적으로 변환됩니다.

In [114]:
df['horsepower'] = df['horsepower'].astype('float')
df['horsepower'].dtypes

dtype('float64')

다음은 'origin' 열의 값을 바꾸겠습니다. 데이터셋 속성에 대한 설명을 보면 'origin' 열의 1은 USA, 2는 EU, 3은 JPN으로 명시되어 있습니다. 대응하는 문자열 값으로 치환하겠습니다. 먼저 데이터 유형과 고유값을 확인합니다.

In [115]:
print(df['origin'].dtypes)
print()
print(df['origin'].unique())
print()
print(df['origin'].value_counts())

int64

[1 3 2]

1    245
3     79
2     68
Name: origin, dtype: int64


이제 각 숫자에 대응하는 값으로 치환하겠습니다.

In [116]:
df['origin'].replace({1: 'USA', 2: 'EU', 3: 'JPN'}, inplace=True)

In [117]:
print(df['origin'].unique())

['USA' 'JPN' 'EU']


추가로 고려해볼 점이라면 'origin' 열의 값처럼 일정한 값이 반복되는 데이터는 데이터 형식을 범주형으로 구성하는 편이 좋습니다. 현재 데이터 타입은 문자열인 `object`이며, 변환할 예정인 범주형의 데이터 타입은 `category`입니다. `astype()` 메서드를 사용하겠습니다.

In [118]:
df['origin'] = df['origin'].astype('category')
df['origin'].dtypes

CategoricalDtype(categories=['EU', 'JPN', 'USA'], ordered=False)

'origin' 열뿐만 아니라 'model year' 열도 범주형으로 변환하기에 적합합니다(그대로 유지해도 무방합니다). 연도를 의미하는 숫자형 값으로 이루어진 이 열 또한 'origin' 열처럼 일정한 값이 반복됩니다. 아울러 숫자 크기보다는 시간 순서를 나타냅니다. 먼저 데이터를 확인하겠습니다.

In [119]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA,chevrolet chevelle malibu,7.653
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA,buick skylark 320,6.377
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA,plymouth satellite,7.653
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA,amc rebel sst,6.802
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA,ford torino,7.227


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           392 non-null    float64 
 1   cylinders     392 non-null    int64   
 2   displacement  392 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        392 non-null    float64 
 5   acceleration  392 non-null    float64 
 6   model year    392 non-null    int64   
 7   origin        392 non-null    category
 8   name          392 non-null    object  
 9   kpl           392 non-null    float64 
dtypes: category(1), float64(6), int64(2), object(1)
memory usage: 31.1+ KB


In [121]:
df['model year'].unique()

array([70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82])

In [122]:
df['model year'].value_counts()

73    40
78    36
76    34
82    30
75    30
79    29
70    29
81    28
77    28
72    28
80    27
71    27
74    26
Name: model year, dtype: int64

'model year' 열 값의 데이터 형식을 `category`로 변환하겠습니다.

In [123]:
df['model year'] = df['model year'].astype('category')
df['model year'].dtypes

CategoricalDtype(categories=[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82], ordered=False)

## **1.4. 범주형 데이터**

### **1.4.1. 구간 분할하기**

위에서 잠시 범주형인 `category` 데이터 타입에 대해 다뤘습니다. 이번에는 연속 변수를 더 구체적으로 일정한 구간에 따라 분할(binning)하겠습니다.

In [124]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA,chevrolet chevelle malibu,7.653
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA,buick skylark 320,6.377
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA,plymouth satellite,7.653
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA,amc rebel sst,6.802
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA,ford torino,7.227


In [125]:
df['horsepower'].min()

46.0

In [126]:
df['horsepower'].max()

230.0

'horsepower' 열 값의 최솟값과 최댓값은 각각 46.0, 230.0입니다. 참고로 이 열 값은 엔진 출력을 의미합니다. 임의로 구간을 나눈다면 낮은 출력, 중간 출력, 높은 출력으로 나눌 수 있습니다.

일단 넘파이의 `histogram()` 메서드를 사용해서 구간에 속한 값의 개수를 가진 `values_count` 인스턴스와 경곗값을 가진 `boundary_values` 인스턴스를 만들겠습니다. 세 구간으로 나눌 예정이므로 경곗값은 4개가 생성되어야 합니다. 여기서 생성한 인스턴스 중에 `bin_dividers`는 아래에서 사용할 판다스의 `cut()` 메서드에 파라미터 값으로 지정할 예정입니다. 

In [127]:
values_count, boundary_values = np.histogram(df['horsepower'], bins=3)
# 각 구간에 속한 값의 개수를 출력합니다.
print(values_count)
# 경곗값을 출력합니다.
print(boundary_values)

[257 103  32]
[ 46.         107.33333333 168.66666667 230.        ]


각 구간에 속한 값의 개수와 경곗값 4개가 출력됐습니다.

판다스의 `cut()` 메서드를 사용하겠습니다. 매개변수 `x`에 `df['horsepower']`를 지정하여 'horsepower' 열 값을 적용하고, `bins`에 위에서 생성한 `boundary_values` 인스턴스를 지정하여 경계값을 적용합니다. 아울러 `labels`에 구간 3개만큼 구간명 3개를 적용하기 위해 구간명 3개를 가진 리스트 인스턴스인 `bin_name`을 생성해서 적용합니다. `include_lowest`는 처음 경계값(최솟값)인 46을 포함시킬지 여부를 지정하는 매개변수입니다.

In [128]:
bin_name = ['낮은 출력', '중간 출력', '높은 출력']

# 새로운 열인 'horsepower bin'을 생성합니다.
df['horsepower bin'] = pd.cut(x=df['horsepower'],
                              bins=boundary_values,
                              labels=bin_name,
                              include_lowest=True)

df[['horsepower', 'horsepower bin']].value_counts()

horsepower  horsepower bin
150.0       중간 출력             22
90.0        낮은 출력             20
88.0        낮은 출력             19
110.0       중간 출력             18
100.0       낮은 출력             17
                              ..
103.0       낮은 출력              1
94.0        낮은 출력              1
93.0        낮은 출력              1
91.0        낮은 출력              1
102.0       낮은 출력              1
Length: 93, dtype: int64

'horsepower bin' 열의 데이터 타입을 확인하겠습니다. `cut()` 메서드로 새로운 열을 생성하면 `category`로 지정됩니다.

In [129]:
df['horsepower bin'].dtypes

CategoricalDtype(categories=['낮은 출력', '중간 출력', '높은 출력'], ordered=True)

### **1.4.2. 원핫인코딩 하기**

컴퓨터가 인식할 수 있는 숫자는 수의 크기와 관련없이 0과 1입니다. 이렇게 **특성(feature)**의 존재 여부를 0(특성 없음)과 1(특성 있음)로 표현하는 변수를 **더미 변수(dummy variable)**라고 부르며, 0과 1로만 구성되는 **원핫벡터(one hot vector)**로 바꾸는 작업을 **원핫인코딩(one-hot-encoding)**으로 일컫습니다. 범주형 데이터를 머신러닝을 활용한 데이터 분석에 사용하려면 원핫인코딩 작업을 선행해야 합니다.

판다스의 `get_dummies()` 메서드로 'horsepower bin' 열의 범주형 고윳값을 더미 변수로 변환하겠습니다.

In [130]:
horsepower_dummies = pd.get_dummies(df['horsepower bin'])
print(horsepower_dummies.head(20))
print()
print(horsepower_dummies.dtypes)

    낮은 출력  중간 출력  높은 출력
0       0      1      0
1       0      1      0
2       0      1      0
3       0      1      0
4       0      1      0
5       0      0      1
6       0      0      1
7       0      0      1
8       0      0      1
9       0      0      1
10      0      0      1
11      0      1      0
12      0      1      0
13      0      0      1
14      1      0      0
15      1      0      0
16      1      0      0
17      1      0      0
18      1      0      0
19      1      0      0

낮은 출력    uint8
중간 출력    uint8
높은 출력    uint8
dtype: object


사이킷런의 `preprocessing` 모듈을 활용하여 원핫인코딩을 하겠습니다. 먼저 이 모듈의 `LabelEncoder()` 클래스와 `OneHotEncoder()` 클래스를 인스턴스화하겠습니다.

In [131]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

`fit()` 메서드와 `transform()` 메서드를 결합한 `fit_transform()` 메서드에 'horsepower bin' 열의 데이터를 적용하여 문자열 범주를 숫자형 범주로 바꾸겠습니다. 

In [132]:
onehot_labeled = label_encoder.fit_transform(df['horsepower bin'])
print(onehot_labeled)
print(label_encoder.classes_)

[2 2 2 2 2 1 1 1 1 1 1 2 2 1 0 0 0 0 0 0 0 0 0 2 0 1 1 1 1 0 0 0 0 0 0 0 0
 2 1 2 2 1 1 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 2 2 2 1 2 2 1 0 2 2 2
 2 2 0 0 0 0 0 0 0 0 1 2 2 2 2 1 2 2 2 1 1 1 0 0 0 0 0 0 2 2 1 1 0 0 0 0 0
 0 0 0 2 1 0 0 0 2 2 2 2 1 0 0 0 0 0 0 0 0 2 0 2 2 2 2 2 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 2 2 2 2 0 2 0 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0
 2 2 2 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 2 1 2 2 2 0 0 0 0 0 2 2 2
 2 2 0 0 0 1 1 1 2 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 2 2 2 0 0 0 0 0 0 0 0 2
 2 2 2 2 2 0 0 0 0 0 0 0 0 0 2 2 2 0 0 2 0 0 0 2 2 2 2 2 2 2 2 2 0 0 0 0 0
 2 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0
 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0 0 0 0 0 0 0 0]
['낮은 출력' '높은 출력' '중간 출력']


'낮은 출력'은 0, '높은 출력'은 1, '중간 출력'은 2를 의미합니다. 해당하는 값에 따라 숫자형 범주로 바뀌었습니다.

`reshape()` 메서드를 사용하여 1차원 배열을 2차원 행렬(392, 1)로 바꾸겠습니다. 2차원 행렬로 바꾸는 이유는 `OneHotEncoder()` 클래스에 1차원 배열이 아닌 2차원 행렬을 적용할 수 있기 때문입니다.

In [133]:
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
print(onehot_reshaped[:10])

[[2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]]


2차원 행렬을 희소행렬로 바꾸겠습니다.

In [134]:
onehot_matrix = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_matrix[:10])
print()
print(onehot_matrix.toarray())
print(onehot_matrix.shape)

  (0, 2)	1.0
  (1, 2)	1.0
  (2, 2)	1.0
  (3, 2)	1.0
  (4, 2)	1.0
  (5, 1)	1.0
  (6, 1)	1.0
  (7, 1)	1.0
  (8, 1)	1.0
  (9, 1)	1.0

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
(392, 3)


## **1.5. 데이터 정규화**

각 속성(열) 값의 숫자 크기가 상대적으로 차이가 나면 데이터 분석 결과가 왜곡될 수밖에 없습니다. 예컨대 0 ~ 100의 범주에 든 값과 0 ~ 10의 범주에 든 값을 토대로 데이터를 분석한다면 전자의 범주에 든 값이 매우 큰 영향력을 가집니다. 이러한 차이를 **정규화(normalization)** 작업을 통해 동일한 크기 기준(0 ~ 1 또는 -1 ~ 1)으로 만든다면 값마다 동일한 영향력을 가질 수 있습니다.

동일한 크기 기준으로 만드는 방법은 여러가지가 있습니다. 그 중 하나는 해당 열의 최댓값의 절댓값(이하 최댓값)으로 각 값을 나누는 것입니다. 예컨대 최댓값을 최댓값으로 나누면 1이 되고, 최댓값 이하의 값을 최댓값으로 나누면 1미만 소수가 됩니다. 아래 코드로 설명하겠습니다. 먼저 `describe()` 메서드를 활용하여 데이터프레임의 요약 통계량을 확인하겠습니다. 

In [135]:
df.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,kpl
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,9.967890
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.318213
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,3.826000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,7.227000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,9.672000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,12.329000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,19.812000


각 값을 최댓값으로 나누고 'horsepower' 열 값으로 변환하겠습니다. 최댓값 1을 제외한 모든 값은 0 ~ 1 범위를 갖습니다.

In [66]:
# `df['horsepower']`는 `df.horsepower`와 같습니다. 'horsepower' 열은 데이터프레임이 가진 속성이기 때문입니다.
df['horsepower'] = df['horsepower'] / abs(df['horsepower'].max())
print(df['horsepower'])
print()
display(df.describe())

0      0.565217
1      0.717391
2      0.652174
3      0.652174
4      0.608696
         ...   
393    0.373913
394    0.226087
395    0.365217
396    0.343478
397    0.356522
Name: horsepower, Length: 392, dtype: float64



,mpg,cylinders,displacement,horsepower,weight,acceleration,kpl
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,0.454215,2977.584184,15.541327,9.967890
std,7.805007,1.705783,104.644004,0.167353,849.402560,2.758864,3.318213
min,9.000000,3.000000,68.000000,0.200000,1613.000000,8.000000,3.826000
25%,17.000000,4.000000,105.000000,0.326087,2225.250000,13.775000,7.227000
50%,22.750000,4.000000,151.000000,0.406522,2803.500000,15.500000,9.672000
75%,29.000000,8.000000,275.750000,0.547826,3614.750000,17.025000,12.329000
max,46.600000,8.000000,455.000000,1.000000,5140.000000,24.800000,19.812000


각 값과 최솟값의 차(a)를 최댓값과 최솟값의 차(b)로 나누는 정규화 방식도 있습니다. $a/b$ 계산식으로 정규화하는 것입니다. 0 ~ 1 범위의 값이 도출되지만 처음 제시했던 정규화 방식(각 값을 최댓값으로 나누는 방식)과는 다른 값이 도출됩니다.

In [137]:
a_horsepower = df.horsepower - df.horsepower.min()
b_horsepower = df.horsepower.max() - df.horsepower.min()
df.horsepower = a_horsepower / b_horsepower
print(df['horsepower'])
print()
display(df.describe())

0      0.456522
1      0.646739
2      0.565217
3      0.565217
4      0.510870
         ...   
393    0.217391
394    0.032609
395    0.206522
396    0.179348
397    0.195652
Name: horsepower, Length: 392, dtype: float64



,mpg,cylinders,displacement,horsepower,weight,acceleration,kpl
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,0.317768,2977.584184,15.541327,9.967890
std,7.805007,1.705783,104.644004,0.209191,849.402560,2.758864,3.318213
min,9.000000,3.000000,68.000000,0.000000,1613.000000,8.000000,3.826000
25%,17.000000,4.000000,105.000000,0.157609,2225.250000,13.775000,7.227000
50%,22.750000,4.000000,151.000000,0.258152,2803.500000,15.500000,9.672000
75%,29.000000,8.000000,275.750000,0.434783,3614.750000,17.025000,12.329000
max,46.600000,8.000000,455.000000,1.000000,5140.000000,24.800000,19.812000


## **1.6. 시계열 데이터**

### **1.6.1. 시계열 객체로 변환하기**

### **1.6.2. 시계열 데이터 생성하기**

### **1.6.3. 시계열 데이터 활용하기**